## 결측값(missing value) 처리

- 제거
- 대체(imputation)
    - 자기참조 vs 다른 변수와의 관계 이용
    - deterministic imputation vs stochastic imputation
    - single imputation vs multiple imputation
    
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html


### 행제거: 관측값 제거

- 모든 행이 NA이면 제거
- 특정 변수의 값이 NA이면 제거
- 결측값이 몇 개 이상이면 제거

In [1]:
import pandas as pd
import numpy as np

In [2]:
결측자료 = pd.read_csv('Employee_missing.csv', encoding = 'cp949')
결측자료.head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,사무직,60000.0,NaN,95.0,NaN,YES
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


#### 결측값 표시 변경: read_csv 옵션 중 na_values에서 변경 가능

- 아래의 문자는 모두 NaN으로 처리
    - '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'   
    
- keep_default_na 옵션(default = True)와 연동
    - keep_default_na = True & na_values 미설정: 기존 NaN 표현만 사용
    - keep_default_na = True & na_values 설정: 기존 NaN 표현에 설정된 값 추가
    - keep_default_na = False & na_values 미설정: NaN 처리 문자 없음
    - keep_default_na = False & na_values 설정: 설정된 값만 NaN으로 처리

In [3]:
# 결측자료의 수
결측자료.isna().sum()

id          2
gender      2
educ        2
jobcat      2
salary      2
salbegin    3
jobtime     2
prevexp     3
minority    2
dtype: int64

In [4]:
# 결측값이 하나라도 있는 경우 제거
결측자료.dropna(axis = 0).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No
7,5.0,남성,15.0,사무직,45000.0,21000.0,98.0,138.0,No


In [5]:
# 모든 자료가 결측값이면 제거: how = 'all', default = 'any'
결측자료.dropna(how='all').head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,사무직,60000.0,NaN,95.0,NaN,YES
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No


In [6]:
# 결측값이 아닌 개수가 k개 미만이면 제거
k = 5
결측자료.dropna(thresh = k).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
1,0.0,NaN,NaN,사무직,60000.0,NaN,95.0,NaN,YES
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No


In [7]:
# 결측값이 특정 변수(column)에 있으면 제거
결측자료.dropna(subset = 'gender').head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No


In [8]:
결측자료.dropna(subset = ['gender', 'jobcat']).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No
7,5.0,남성,15.0,사무직,45000.0,21000.0,98.0,138.0,No


In [9]:
# 결측을 제거한 데이터 프레임을 동일명의 데이터 프레임으로 저장
결측자료.dropna(subset = 'gender', inplace = True)
결측자료.head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No


### 열제거: 변수 제거

In [10]:
결측자료.dropna(axis = 1).head()

,gender,educ
0,남성,12.0
3,남성,15.0
4,남성,16.0
5,여성,12.0
6,여성,8.0


In [11]:
결측자료.dropna(axis = 'columns').head()

,gender,educ
0,남성,12.0
3,남성,15.0
4,남성,16.0
5,여성,12.0
6,여성,8.0


In [12]:
결측자료.dropna(axis = 1, thresh = 474).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No
5,3.0,여성,12.0,사무직,21450.0,12000.0,98.0,381.0,No
6,4.0,여성,8.0,사무직,21900.0,13200.0,98.0,190.0,No


In [13]:
결측자료.dropna(axis = 1, thresh = 475).head()

,gender,educ
0,남성,12.0
3,남성,15.0
4,남성,16.0
5,여성,12.0
6,여성,8.0


In [14]:
# 특정변수의 값을 NaN으로 처리
결측자료['jobcat'].replace('경영자', np.NaN)

0      NaN
3      NaN
4      사무직
5      사무직
6      사무직
      ... 
472    사무직
473    사무직
474    사무직
475    사무직
476    사무직
Name: jobcat, Length: 475, dtype: object

In [15]:
결측자료['jobcat'].replace({'경영자':np.NaN, '사무직':np.NaN})

0      NaN
3      NaN
4      NaN
5      NaN
6      NaN
      ... 
472    NaN
473    NaN
474    NaN
475    NaN
476    NaN
Name: jobcat, Length: 475, dtype: object

## 결측값 대체 (imputation)
- 대체(imputation)
    - 자기참조 vs 다른 변수와의 관계 이용
    - deterministic imputation vs stochastic imputation
    - single imputation vs multiple imputation
    
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

In [16]:
결측자료 = pd.read_csv('Employee_missing.csv', encoding = 'cp949')
결측자료.head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,사무직,60000.0,NaN,95.0,NaN,YES
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


In [17]:
# 대체값 직접 지정
결측자료.fillna(0).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0.0,남성,12.0,0,0.0,0.0,0.0,0.0,0
1,0.0,0,0.0,사무직,60000.0,0.0,95.0,0.0,YES
2,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


In [18]:
결측자료.fillna({'id':0, 'gender':'남자', 'jobcat':'경영자'}).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0.0,남성,12.0,경영자,NaN,NaN,NaN,NaN,NaN
1,0.0,남자,NaN,사무직,60000.0,NaN,95.0,NaN,YES
2,0.0,남자,NaN,경영자,NaN,NaN,NaN,NaN,NaN
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


In [19]:
# 전/후 관측값으로 대체(관측값의 순서가 시간순인 경우)
결측자료.fillna(method = 'ffill').head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,NaN,남성,12.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,남성,12.0,사무직,60000.0,NaN,95.0,NaN,YES
2,0.0,남성,12.0,사무직,60000.0,NaN,95.0,NaN,YES
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


In [20]:
결측자료.fillna(method = 'bfill').head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0.0,남성,12.0,사무직,60000.0,27000.0,95.0,144.0,YES
1,0.0,남성,15.0,사무직,60000.0,27000.0,95.0,144.0,YES
2,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


### 특정 통계값으로 대체
- 수치자료: 평균, 중앙값, ...
- 범주자료: 최빈값, ...

In [21]:
수치변수명 = 결측자료.select_dtypes(include = np.number).columns.tolist()
수치결측자료 = 결측자료[수치변수명]
수치결측자료.head()

,id,educ,salary,salbegin,jobtime,prevexp
0,NaN,12.0,NaN,NaN,NaN,NaN
1,0.0,NaN,60000.0,NaN,95.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,15.0,57000.0,27000.0,98.0,144.0
4,2.0,16.0,40200.0,18750.0,98.0,36.0


In [22]:
수치결측자료.mean()

id            237.000000
educ           13.488421
salary      34473.421053
salbegin    17016.086498
jobtime        81.138947
prevexp        95.860759
dtype: float64

In [23]:
수치결측자료.fillna(수치결측자료.mean()).head()
# 중앙값: median(), 최솟값: min(), 최댓값: max()

,id,educ,salary,salbegin,jobtime,prevexp
0,237.0,12.000000,34473.421053,17016.086498,81.138947,95.860759
1,0.0,13.488421,60000.000000,17016.086498,95.000000,95.860759
2,237.0,13.488421,34473.421053,17016.086498,81.138947,95.860759
3,1.0,15.000000,57000.000000,27000.000000,98.000000,144.000000
4,2.0,16.000000,40200.000000,18750.000000,98.000000,36.000000


In [24]:
최빈값 = 결측자료.mode()
최빈값

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0.0,남성,12.0,사무직,30750.0,15000.0,81.0,0.0,No
1,1.0,NaN,NaN,NaN,NaN,NaN,93.0,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
470,470.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,471.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,472.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,473.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
결측자료.fillna(최빈값.iloc[0, :]).head()

,id,gender,educ,jobcat,salary,salbegin,jobtime,prevexp,minority
0,0.0,남성,12.0,사무직,30750.0,15000.0,81.0,0.0,No
1,0.0,남성,12.0,사무직,60000.0,15000.0,95.0,0.0,YES
2,0.0,남성,12.0,사무직,30750.0,15000.0,81.0,0.0,No
3,1.0,남성,15.0,경영자,57000.0,27000.0,98.0,144.0,No
4,2.0,남성,16.0,사무직,40200.0,18750.0,98.0,36.0,No


### 대체 패키지

https://scikit-learn.org/stable/modules/impute.html

<br>

- Scikit-learn: Simplelmputer
    - https://scikit-learn.org/stable/modules/generated/sklearn.impute.Simplelmputer.html
- Scit-learn: Iterativelmputer
    - A strategy for imputing missing values by modeling each feature with missing values as a function of other features in a round-robin fashion
    - https://scikit-learn.org/stable/modules/generated/sklearn.impute.Iterativelmputer.html
- MICE

In [26]:
from sklearn.impute import SimpleImputer

단순대체 = SimpleImputer(strategy = 'mean')
대체자료 = 단순대체.fit_transform(수치결측자료)
# 평균 = mean, 중앙값 = medien, 최빈값 = most_frequent, 정해진 값 = constant (fiflvalue로 2 지정)
대체자료

array([[2.37000000e+02, 1.20000000e+01, 3.44734211e+04, 1.70160865e+04,
        8.11389474e+01, 9.58607595e+01],
       [0.00000000e+00, 1.34884211e+01, 6.00000000e+04, 1.70160865e+04,
        9.50000000e+01, 9.58607595e+01],
       [2.37000000e+02, 1.34884211e+01, 3.44734211e+04, 1.70160865e+04,
        8.11389474e+01, 9.58607595e+01],
       ...,
       [4.72000000e+02, 1.50000000e+01, 3.91500000e+04, 1.57500000e+04,
        6.30000000e+01, 4.60000000e+01],
       [4.73000000e+02, 1.20000000e+01, 2.14500000e+04, 1.27500000e+04,
        6.30000000e+01, 1.39000000e+02],
       [4.74000000e+02, 1.20000000e+01, 2.94000000e+04, 1.42500000e+04,
        6.30000000e+01, 9.00000000e+00]])

In [27]:
대체자료 = pd.DataFrame(대체자료, columns=수치결측자료.columns)
대체자료.head()

,id,educ,salary,salbegin,jobtime,prevexp
0,237.0,12.000000,34473.421053,17016.086498,81.138947,95.860759
1,0.0,13.488421,60000.000000,17016.086498,95.000000,95.860759
2,237.0,13.488421,34473.421053,17016.086498,81.138947,95.860759
3,1.0,15.000000,57000.000000,27000.000000,98.000000,144.000000
4,2.0,16.000000,40200.000000,18750.000000,98.000000,36.000000


In [28]:
단순대체.set_output(transform='pandas')
대체자료 = 단순대체.fit_transform(수치결측자료)
대체자료.head()

,id,educ,salary,salbegin,jobtime,prevexp
0,237.0,12.000000,34473.421053,17016.086498,81.138947,95.860759
1,0.0,13.488421,60000.000000,17016.086498,95.000000,95.860759
2,237.0,13.488421,34473.421053,17016.086498,81.138947,95.860759
3,1.0,15.000000,57000.000000,27000.000000,98.000000,144.000000
4,2.0,16.000000,40200.000000,18750.000000,98.000000,36.000000


In [29]:
from sklearn.experimental import enable_iterative_imputer # noqa
from sklearn.impute import IterativeImputer

In [30]:
확률대체 = IterativeImputer(random_state = 0)
확률대체.set_output(transform = 'pandas')
대체자료 = 확률대체.fit_transform(수치결측자료)
대체자료.head()

,id,educ,salary,salbegin,jobtime,prevexp
0,237.299536,12.000000,34023.646203,17045.190569,81.132509,116.751862
1,0.000000,16.290908,60000.000000,26325.929668,95.000000,88.290084
2,237.000629,13.494309,34472.476147,17035.706135,81.138934,95.888744
3,1.000000,15.000000,57000.000000,27000.000000,98.000000,144.000000
4,2.000000,16.000000,40200.000000,18750.000000,98.000000,36.000000


In [34]:
# KNN
from sklearn.impute import KNNImputer

근접대체 = KNNImputer(n_neighbors = 190, weights = 'uniform')   # educ=12인 데이터 (0) - educ=12인 데이터 190개의 평균으로 결측치 채움
근접대체.set_output(transform = 'pandas')
근접대체.fit_transform(수치결측자료).head()

,id,educ,salary,salbegin,jobtime,prevexp
0,243.763158,12.000000,25887.157895,13241.868421,80.768421,96.378947
1,0.000000,15.200000,60000.000000,21198.526316,95.000000,91.168421
2,237.000000,13.488421,34473.421053,17016.086498,81.138947,95.860759
3,1.000000,15.000000,57000.000000,27000.000000,98.000000,144.000000
4,2.000000,16.000000,40200.000000,18750.000000,98.000000,36.000000


In [32]:
수치결측자료.groupby('educ').count()

,id,salary,salbegin,jobtime,prevexp
educ,,,,,
8.0,53,53,53,53,53
12.0,190,190,190,190,190
14.0,6,6,6,6,6
15.0,116,116,116,116,116
16.0,59,59,59,59,59
17.0,11,11,11,11,11
18.0,9,9,9,9,9
19.0,27,27,27,27,27
20.0,2,2,2,2,2


In [33]:
수치결측자료.groupby('educ').mean()

,id,salary,salbegin,jobtime,prevexp
educ,,,,,
8.0,256.528302,24399.056604,13064.150943,79.773585,183.245283
12.0,243.763158,25887.157895,13241.868421,80.768421,96.378947
14.0,257.833333,31625.000000,15625.000000,79.666667,57.333333
15.0,222.387931,31685.000000,15610.603448,82.120690,77.784483
16.0,224.237288,48225.932203,22338.474576,82.016949,62.745763
17.0,218.454545,59527.272727,26904.545455,82.272727,104.454545
18.0,244.777778,65127.777778,32240.000000,80.666667,82.444444
19.0,260.888889,72520.370370,34764.074074,79.111111,75.925926
20.0,151.500000,64312.500000,36240.000000,87.000000,70.000000


### 회귀대체
- 결측값이 없는 설명변수와 반응변수 자료로 회귀모형 적합
- 결측값이 있는 설명변수에 대해 예측값 계산
    - t-분포 난수와 표준오차를 곱한 오차를 예측값에 더함
    - np.random.standard_t(자유도, 표본크기)
- 예측값을 반응변수에 대입
    - df['반응변수'].fillna(pd.Series(예측값.flatten()).inplace = True)